In [11]:
import numpy as np
import cv2 as cv
import csv 
import tensorflow as tf
from keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Dense,Input,Concatenate,Resizing,Reshape,Flatten
from keras import Model


In [12]:
csv_ds = 'dataset\\image_csv\\ds_images.csv'

In [13]:
csv_data = []

with open(csv_ds,'r',encoding='utf-8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        if len(row)>0:
            csv_data.append(row)
ids = [i for i in range(len(csv_data))]

In [14]:
def string_to_tuple(string_data,normalize = (1,1)):
    list_data = string_data.split(',')
    int_val1 = int(list_data[0]) / normalize[0]
    int_val2 = int(list_data[1]) / normalize[1]
    
    return (round(int_val1, 3),round(int_val2, 3))

In [15]:
def load_data(ids):
    images  = []
    lables = []
    for id in ids:
        image_location =csv_data[id][0]
        img = cv.imread(image_location)
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)/255.0
        img = cv.resize(img,(img.shape[0]//2,img.shape[1]//2),interpolation = cv.INTER_LANCZOS4)
        img_p1 = string_to_tuple(csv_data[id][3],img.shape)
        img_p2 = string_to_tuple(csv_data[id][4],img.shape)
        name_p1 = string_to_tuple(csv_data[id][5],img.shape)
        name_p2 = string_to_tuple(csv_data[id][6],img.shape)
        id_p1 = string_to_tuple(csv_data[id][7],img.shape)
        id_p2 = string_to_tuple(csv_data[id][8],img.shape)
        lables.append(np.array([img_p1[0],img_p1[1],img_p2[0],img_p2[1],name_p1[0],name_p1[1],name_p2[0],name_p2[1],id_p1[0],id_p1[1],id_p2[0],id_p2[1]]))
        images.append(img.reshape((640, 410,1)))

    return np.array(images),np.array(lables)


In [16]:
def batch_generator(ids, batch_size = 32):
    batch=[]
    while True:
            np.random.shuffle(ids) 
            for i in ids:
                batch.append(i)
                if len(batch)==batch_size:
                    yield load_data(batch)
                    batch=[]
            break

In [17]:
def object_detection_model():
    inputs = Input(shape=(640, 410,1)) #TODO: resize
    # TODO: reshape 
    # feature extraction
    x = Conv2D(8,(7,7) ,activation= 'relu')(inputs)
    x= BatchNormalization()(x)
    x= MaxPooling2D((3,3))(x)

    x = Conv2D(8,(7,7) ,activation= 'relu')(x)
    x= BatchNormalization()(x)
    x= MaxPooling2D((3,3))(x)

    x = Conv2D(32,(3,3) ,activation= 'relu')(x)
    x= BatchNormalization()(x)
    x= MaxPooling2D((3,3))(x)
    # multy head
        ## image head
    img = Conv2D(64,(3,3),activation='relu')(x)
    img= BatchNormalization()(img)
    img= MaxPooling2D((3,3))(img)
    img = Flatten()(img)
    img = Dense(8, 'relu')(img)
    img = Dense(4,'relu')(img)
        ## name head 
    name = Conv2D(64,(3,3),activation='relu')(x)
    name= BatchNormalization()(name)
    name= MaxPooling2D((3,3))(name)
    name = Flatten()(name)

    name = Dense(16, 'relu')(name)
    name = Dense(4,'relu')(name)

        ## id head
    id = Conv2D(64,(3,3),activation='relu')(x)
    id= BatchNormalization()(id)
    id= MaxPooling2D((3,3))(id)
    id = Flatten()(id)
    id = Dense(16, 'relu')(id)
    id = Dense(4,'relu')(id)
    # concatendate output
    conc = Concatenate()([img,name,id])
    output = Dense(12,'sigmoid')
    return Model(inputs= inputs,outputs =conc ,name= 'id_card_detection')

In [18]:
model = object_detection_model()


In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='mse',metrics=['accuracy'])

In [22]:
model.fit(batch_generator(ids),epochs=100)

Epoch 1/100
162/162 [==============================] - 94s 582ms/step - loss: 0.3120 - accuracy: 0.5600
Epoch 2/100
162/162 [==============================] - 0s 160us/step - loss: 0.3120 - accuracy: 0.5600
